# Метод косинусного сходства

## О методе

Метод **косинусного сходства** (*cosine similarity*) в коллаборативной фильтрации используется для измерения сходства между пользователями или предметами на основе их характеристик или предпочтений.

Косинусное сходство измеряет **угол** между двумя **нормированными** векторами в многомерном пространстве.

- Если угол маленький (векторы направлены в одну сторону), сходство близко к 1 (*почти идентичные предпочтения*)
- Если угол большой (векторы направлены в разные стороны), сходство близко к 0 (*нет связи*)
- Если векторы полностью противоположны, сходство -1 (*полная антипатия, но такое можно встретить крайне редко, отрицательные значения можно, например, обнулять или брать по модулю, рекомендательной системе важнее выявить сходства, чем противоположные предпочтения*)

Косинус угла $ \theta $ между векторами $ \mathbf{A} $ и $ \mathbf{B} $ находится по формуле:

$$
\cos \theta = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}
$$

где:

- $ \mathbf{A} \cdot \mathbf{B} = \sum_{i=1}^{n} A_i B_i = A_1 B_1 + A_2 B_2 + \dots + A_n B_n $ — скалярное произведение векторов,
- $ \|\mathbf{A}\| $ и $ \|\mathbf{B}\| $ — их длины (модули), вычисляемые как:

$$
\|\mathbf{A}\| = \sqrt{\sum_{i=1}^{n} A_i^2} = \sqrt{A_1^2 + A_2^2 + \dots + A_n^2}, \|\mathbf{B}\| = \sqrt{\sum_{i=1}^{n} B_i^2} = \sqrt{B_1^2 + B_2^2 + \dots + B_n^2}
$$

## Реализация

### Данные

**MIND** (***Mi**crosoft* ***N**ews* ***D**ataset*) - крупномасштабный набор данных для исследования рекомендаций новостей. Содержит около 160 тыс. новостных статей на английском языке и более 15 млн. журналов показов, созданных 1 млн. пользователей.

Будет использоваться **MIND-small** - около 50 тыс. пользователей и их журналы взаимодействия.

Данные состоят из нескольких файлов, среди которых интересующий нас - журнал взаимодействия пользователей (*behaviors.tsv*)

Файл содержит таблицу, каждая строка которой состоит из номера взаимодействия, id пользователя, даты и времени взаимодействия, показанных пользователю новостей, новостей, на которые пользователь кликнул.

In [17]:
import pandas as pd

df = pd.read_csv('../../data/Mindsmall_train/behaviors.tsv', sep='\t', header=None, names=['id', 'user_id', 'timestamp', 'shown_news', 'clicked_news'])

print(df.head(2))

   id user_id              timestamp  \
0   1  U13740  11/11/2019 9:05:58 AM   
1   2  U91836  11/12/2019 6:11:30 PM   

                                          shown_news  \
0  N55189 N42782 N34694 N45794 N18445 N63302 N104...   
1  N31739 N6072 N63045 N23979 N35656 N43353 N8129...   

                                        clicked_news  
0                                  N55689-1 N35729-0  
1  N20678-0 N39317-0 N58114-0 N20495-0 N42977-0 N...  


### Идея

Для реализации метода косинусного сходства нужны две вещи: векторы и сущности, которые они представляют (в нашем случае - пользователи). В качестве вектора можно представить список новостей, на которые пользователь кликнул.

Возможных значений координат вектора всего два:

- 1 - если пользователь кликнул на новость
- 0 - если не кликнул

### Код

In [18]:
import numpy as np

In [ ]:
def cosine_similarity(user1, user2):
    # Ищем общие элементы (например, статьи, которые оба пользователя читали)
    mask = ~np.isnan(user1) & ~np.isnan(user2)
    if np.sum(mask) == 0:
        return 0  # Если нет общих оценок, сходство = 0
    user1_filtered = user1[mask]
    user2_filtered = user2[mask]
    
    # Косинусное сходство
    numerator = np.dot(user1_filtered, user2_filtered)
    denominator = np.linalg.norm(user1_filtered) * np.linalg.norm(user2_filtered)
    return numerator / denominator

def calculate_user_similarity(df):
    similarity_matrix = np.zeros((df.shape[0], df.shape[0]))  # Создаем пустую матрицу схожести
    
    for i in range(df.shape[0]):
        for j in range(i + 1, df.shape[0]):  # Ищем только уникальные пары (i, j)
            similarity = cosine_similarity(df.iloc[i], df.iloc[j])  # Вычисляем сходство
            similarity_matrix[i, j] = similarity
            similarity_matrix[j, i] = similarity  # Симметричная матрица
            
    return pd.DataFrame(similarity_matrix, index=df.index, columns=df.index)